In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import torch
from learning_dynamics.data_modules import FingerReconDataModule
from learning_dynamics.models import FingerPEGPVAEModel, Encoder, Decoder
from learning_dynamics.callbacks import FingerPEGPVAEPlotting
import wandb
from lightning.pytorch.callbacks import ModelCheckpoint
import lightning as L
torch.set_default_dtype(torch.float64)

In [ ]:
train_data_path = '../../../../data/ODEs/finger/real_finger/pickle_files/shortened_real_finger_pos_time_1.5_ls_0.6_mean_22_var_4_scale_100.pkl'

In [ ]:
batch_size = 32
data_module = FingerReconDataModule(train_data_path=train_data_path, batch_size=batch_size, seed=0)
data_module.setup("fit")

In [ ]:
# params
width = data_module.train_dataset.dataset.width
height = data_module.train_dataset.dataset.height
embed_dim = 128
latent_dim = 1
dt = data_module.train_dataset.dataset.dt
val_check = 10
dq = 90
cq = 0.01
ls = -0.2
file = f"video_physics"
max_epochs = 6000

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath=f"model_checkpoints/{file}",          
    filename="model-{epoch:03d}",  
    save_top_k=-1,                 
    every_n_epochs=val_check,               
    save_on_train_epoch_end=False,
    save_last=True
)

In [ ]:
# ckpt_path = f"model_checkpoints/{file}/model-epoch=19999.ckpt" 
# trainer = L.Trainer(devices=[1], max_epochs=max_epochs, callbacks=[FingerPEGPVAEPlotting(dt=dt, file=file, vids_per_batch=3), checkpoint_callback], check_val_every_n_epoch=val_check, logger=False)
# trainer.fit(model, datamodule=data_module, ckpt_path="last")

In [ ]:
ckpt_path = f"model_checkpoints/{file}/model-epoch=649.ckpt" 

encoder = Encoder(width, height, embed_dim, latent_dim)
decoder = Decoder(width, height, embed_dim, latent_dim)
#sum
trainer = L.Trainer(devices=[1], max_epochs=max_epochs, callbacks=[FingerPEGPVAEPlotting(dt=dt, file=file, vids_per_batch=0), checkpoint_callback], check_val_every_n_epoch=val_check, logger=False)
model = FingerPEGPVAEModel(encoder, decoder, dt, dq=dq, cq=cq, ls=ls, amp=False, scale=False)
trainer.fit(model, datamodule=data_module)